In [1]:
from selenium import webdriver
from datetime import datetime
import time

In [10]:
# url variable must be url for "customer reviews" page of product.

urls = ["https://www.amazon.com/23andMe-DNA-Test-Ancestry-Personal/product-reviews/B01G7PYQTM/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews",
        "https://www.amazon.com/HEALTH-ANCESTRY-TellmeGen-Online-Reports/product-reviews/B01A0W1D9E/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews",
        "https://www.amazon.com/Color-Conditions-Medication-Including-Available/product-reviews/B07CH39WVS/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews",
        "https://www.amazon.com/Color-Hereditary-Cancer-Test-Including/product-reviews/B01N9VH79S/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews",
        "https://www.amazon.com/Vitagene-DNA-Test-Kit-Ancestry/product-reviews/B01NCISWQT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews",
        "https://www.amazon.com/CellMaxLife-DNA-Cancer-Risk-Test/product-reviews/B074XC1HDB/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
       ]
names = ["23andMe", "TellmeGen", "Color Genetic Risk Test", "Color Heriditary Cancer Test", "CellMaxLife"]


In [ ]:
def get_reviews(driver, name, url, reviews_data):
    """
    Takes a list of review data and adds all reviews for a certain Amazon product to it.
    """
    reviews = driver.find_elements_by_class_name("review")
    for i, review in enumerate(reviews):
        rating = float(review.find_element_by_class_name("review-rating") \
                             .find_element_by_tag_name("span") \
                             .get_attribute("innerHTML") \
                             .split(' ')[0])  # get rating out of 5 stars
        date_str = review.find_element_by_class_name("review-date").text
        date = str(datetime.strptime(date_str, "%B %d, %Y"))
        user_name = review.find_element_by_class_name("a-profile-name").get_attribute("innerHTML")
        try:
            votes = review.find_element_by_class_name("cr-vote-text").get_attribute("innerHTML").split(' ')[0].replace(',','')
            if votes == 'One':
                votes = 1
            else:
                votes = int(review.find_element_by_class_name("cr-vote-text").get_attribute("innerHTML").split(' ')[0].replace(',',''))
        except:
            votes = 0
        review_text = review.find_element_by_class_name("review-text").text
        review_title = review.find_element_by_class_name("review-title").text

        reviews_data.append({'name': name,
                             'id': '',
                             'title': review_title,
                             'author_name': user_name,
                             'author_uri': '',
                             'voteSum': '',
                             'voteCount': votes,
                             'rating': rating,
                             'text': review_text,
                             'date': date})
    return reviews_data

In [ ]:
reviews_data = []

for name, url in zip(names, urls):
    driver = webdriver.Chrome('/usr/local/bin/chromedriver') 
    driver.get(url)
    try:
        next_page_btn = driver.find_element_by_class_name("a-pagination") \
                              .find_element_by_class_name("a-last")
    except:
        next_page_btn = None
        
    if next_page_btn is not None:
        while('a-disabled' not in next_page_btn.get_attribute("class")):
            reviews_data = get_reviews(driver, name, url, reviews_data)
            next_page_btn.click()
            time.sleep(1.5)
            next_page_btn = driver.find_element_by_class_name("a-pagination") \
                                  .find_element_by_class_name("a-last")
    else:
        reviews_data = get_reviews(driver, name, url, reviews_data)

0
100
0
0
0


In [99]:
import pandas as pd
df = pd.DataFrame(reviews_data, 
                  columns=['name', 'id', 'title', 'author_name', 'author_uri', 
                           'voteSum', 'voteCount', 'rating', 'text', 'date']
                 )
df.head()

In [101]:
df.to_csv("data/amazon_all_reviews.csv", index=False)